## ML-第三版本-类别特征编码-交叉累计特征
### 结果记录
+ 交叉累计特征+lightgbm: 训练集AUC均值：0.9386216031892072, 训练集合AUC方差: 0.001409275104703724
+ 交叉累计特征+分箱+lightgbm: 训练集AUC均值：0.9997557027280817, 训练集合AUC方差: 6.28066104293635e-05
+ 类别特征+交叉累计特征+lightgbm: 训练集AUC均值：0.9417167997112633, 训练集合AUC方差: 0.0016311908434587204
+ 类别特征+交叉累计特征WOE+lightgbm: 训练集AUC均值：0.9997050271838572, 训练集合AUC方差: 0.00013631547061740645

### 结论
+ 和第二版的结论一致，考虑一下之前的woe分箱为什么没有这个效果

## 导包

In [2]:
import os
import re
import numpy as np
import pandas as pd
import datetime
import pickle as pkl

import matplotlib.pyplot as plt
import seaborn as sns


import scorpyo as sp
from null_importance import get_null_importance
from time_sequence_feats import get_time_base, get_sequence_statis, get_sequence_groupby_statis
import category_encoders as ce

from gensim.models import word2vec



import time
from sklearn.metrics import roc_auc_score as auc
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold

import warnings
warnings.filterwarnings("ignore")

pd.set_option('max_rows', 320, 'max_columns',100)

In [24]:
path_project = r'/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification'

# path dir
path_row_data = os.path.join(path_project, 'row_data')
path_new_data = os.path.join(path_project, 'new_data')
path_results  = os.path.join(path_project, 'results')
path_results_version  = os.path.join(path_results, 'version_2')

# path row_data
path_train = os.path.join(path_row_data, 'train.csv')
path_test  = os.path.join(path_row_data, 'evaluation_public.csv')
path_sample_submission = os.path.join(path_row_data, 'submit_example.csv')

## results
path_output_report = os.path.join(path_results, '01_原始数据探察_20221014.xlsx')

y_label = "is_risk"


# new_train_data
path_new_train_cate = os.path.join(path_new_data, 'train_cate.pkl')
path_new_test_cate  = os.path.join(path_new_data, 'test_cate.pkl')

path_new_train_time_sequence = os.path.join(path_new_data, 'train_time_sequence.pkl')
path_new_test_time_sequence  = os.path.join(path_new_data, 'test_time_sequence.pkl')

path_new_train_time_sequence_woe = os.path.join(path_new_data, 'train_time_sequence_woe.pkl')
path_new_test_time_sequence_woe  = os.path.join(path_new_data, 'test_time_sequence_woe.pkl')

path_new_train_cumsum = os.path.join(path_new_data, 'train_cumsum.pkl')
path_new_test_cumsum = os.path.join(path_new_data, 'test_cumsum.pkl')

path_new_train_cumsum_woe = os.path.join(path_new_data, 'train_cumsum_woe.pkl')
path_new_test_cumsum_woe = os.path.join(path_new_data, 'test_cumsum_woe.pkl')

In [4]:
df_row_train = pd.read_csv(path_train)
df_row_val  = pd.read_csv(path_test)

## 工具函数

In [5]:
def save_pkl(data, path_data):
    """将数据保存为pkl"""
    with open(path_data, 'wb') as f:
        pkl.dump(data, f)
        
def read_pkl(path_data):
    """读取pkl格式数据"""
    with open(path_data, 'rb') as f:
        data = pkl.load(f)
    return data

## 交叉累计特征

In [6]:
df = pd.concat([df_row_train, df_row_val]).reset_index(drop=True)
df = df.sort_values(by='op_datetime')

# 认证日期时间
df['op_datetime'] = pd.to_datetime(df['op_datetime'])
# 将数据分为每一天
df['op_days'] = df['op_datetime'].map(lambda x: x.strftime('%Y-%m-%d'))

# 按客户进行统计这次认证和上次认证的时间差
df['op_second'] = df['op_datetime']
df['op_second1'] = df.groupby('device_num_transform')['op_second'].shift(1)
df['op_diff_second1'] = (df['op_second'] - df['op_second1']).map(lambda x: x.total_seconds())

df['op_diff_second1_tmp'] = df['op_diff_second1']>10
# 客户第几次登录
df['op_times_groups'] = df.groupby('device_num_transform')['op_diff_second1_tmp'].apply(lambda x: x.cumsum())

# 系统层面的一段时间
df['system_op_second'] = df['op_datetime']
df['system_op_second1'] = df['system_op_second'].shift(1)
df['system_op_diff_second1'] = (df['system_op_second'] - df['system_op_second1']).map(lambda x: x.total_seconds())

df['system_op_diff_second1_tmp'] = df['system_op_diff_second1']>400
# 客户第几次登录
df['system_op_times_groups'] = df['system_op_diff_second1_tmp'].cumsum()

df = df.drop(columns=['op_second','op_second1','op_diff_second1_tmp', 'op_diff_second1', 'system_op_diff_second1',
                      'system_op_second', 'system_op_second1', 'system_op_diff_second1_tmp'])

In [7]:
df.head()

,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,op_datetime,ip_type,http_status_code,op_city,log_system_transform,url,op_month,is_risk,op_days,op_times_groups,system_op_times_groups
44477,44477,xiongkai3397,rd,6H1iPLgBB,GCgxrFb69up7,chrome_93,chrome,win,win10,2022-01-07 02:44:29,内网,200,深圳,nHrKgKdJ1Mzt,xxx.com/github,2022-01,1.0,2022-01-07,0,0
45489,45489,zhengguiying7117,rd,0mjaEf4SB,8ftsXFm5I1Ej,safari_13,safari,macos,macos_big_sur_11,2022-01-07 02:54:32,内网,200,成都,nHrKgKdJ1Mzt,xxx.com/github,2022-01,1.0,2022-01-07,0,1
45706,45706,yuanjun5870,hr,1Vk2kEa4X,W1Cstajd8x1s,firefox_78,firefox,win,win7,2022-01-07 03:00:56,内网,200,深圳,a5G25puBl9xj,hr.xxx.com/,2022-01,1.0,2022-01-07,0,1
45901,45901,zhoutingting3694,rd,4Wj6uxLx3,H8NAVsdws95G,edge_93,edge,win,win10,2022-01-07 04:29:34,内网,200,杭州,nHrKgKdJ1Mzt,xxx.com/github,2022-01,1.0,2022-01-07,0,2
43827,43827,yanglin6562,sales,eK12oQmm8,GnkVqPSy5nnl,ie_9,ie,win,win10,2022-01-07 05:17:44,内网,200,重庆,sW0whYIx8LFM,work.xxx.com/task,2022-01,1.0,2022-01-07,0,3


In [8]:
df.columns

Index(['id', 'user_name', 'department', 'ip_transform', 'device_num_transform',
       'browser_version', 'browser', 'os_type', 'os_version', 'op_datetime',
       'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url',
       'op_month', 'is_risk', 'op_days', 'op_times_groups',
       'system_op_times_groups'],
      dtype='object')

### 系统层面

In [9]:

time_feats = ['system_op_times_groups', 'op_days', 'op_month']

cate_feats = ['ip_transform', 'user_name', 'device_num_transform', 'department', 'browser_version', 'browser', 'os_type','os_version',
              'ip_type','http_status_code', 'op_city', 'log_system_transform', 'url']

# 客户+时间+按时间cumsum/cumunique
# 累计量统计
df['helper'] = 1
# 是否为异常状态码
df['http_status_code_helper'] = df['http_status_code'].map(lambda x: 1 if x in [400, 500, 502, 404] else 0) 
# 给样本编号
df['sampler_index_helper'] = df['helper'].cumsum()

for i in time_feats:
    i_tmp = df.groupby([i])
    # 系统往前看，处理了多少事
    df['system_{}_cumsum'.format(i)] = i_tmp['helper'].cumsum()
    # 系统往前看，处理了多少坏事
    df['system_{}_error_code_cumsum'.format(i)] = i_tmp['http_status_code_helper'].cumsum()
    
    for j in cate_feats:
        index_set = set(df.groupby([i, j],as_index=False).first()['sampler_index_helper'].values)
        df['tmp_helper'] = df['sampler_index_helper'].map(lambda x: 1 if x in index_set else 0)
        j_tmp = df.groupby([i, j])
        # 系统往前看，不同维度的处理了多少情况
        df['system_{}_{}_cumunique'.format(i, j)] = j_tmp['tmp_helper'].cumsum()
    

        if j not in ['ip_transform', 'user_name', 'device_num_transform']:
            for k in df[j].unique():
                tmp = df[df[j]==k].groupby([i])

                # 系统往前看，不同维度不同情况分别处理了多少次
                df['system_{}_{}_{}_cumsum'.format(i,j,k)] = tmp['helper'].cumsum()
                # 系统往前看，不同维度不同情况error_code分别处理了多少次
                df['system_{}_{}_{}_error_code_cumsum'.format(i,j,k)] = tmp['http_status_code_helper'].cumsum()

remove_cols = [x for x in df.columns if x[-6:]=='helper']

df = df.drop(columns=remove_cols)

### 个体层面

In [10]:

time_feats = ['op_times_groups', 'op_days', 'op_month']

object_feats = ['ip_transform', 'user_name', 'device_num_transform']

cate_feats = ['ip_transform', 'user_name', 'device_num_transform', 'department', 'browser_version', 'browser', 'os_type','os_version',
              'ip_type','http_status_code', 'op_city', 'log_system_transform', 'url']

# 客户+时间+按时间cumsum/cumunique
# 累计量统计
df['helper'] = 1
# 是否为异常状态码
df['http_status_code_helper'] = df['http_status_code'].map(lambda x: 1 if x in [400, 500, 502, 404] else 0) 
# 给样本编号
df['sampler_index_helper'] = df['helper'].cumsum()


for i in time_feats:
    for j in object_feats:
        j_tmp = df.groupby([i,j])
        df['{}_{}_cumsum'.format(i,j)] = j_tmp['helper'].cumsum()
        df['{}_{}_error_code_cumsum'.format(i,j)] = j_tmp['http_status_code_helper'].cumsum()
        
        for k in cate_feats:
            if k == j: continue
            index_set = set(df.groupby([i,j,k], as_index=False).first()['sampler_index_helper'].values)            
            df['tmp_helper'] = df['sampler_index_helper'].map(lambda x: 1 if x in index_set else 0)
            k_tmp = df.groupby([i,j,k])
            df['{}_{}_{}_cumunique'.format(i,j,k)] = k_tmp['tmp_helper'].cumsum()

            if k not in ['ip_transform', 'user_name', 'device_num_transform']:
                for v in df[k].unique():
                    v_tmp = df[df[k]==v].groupby([i,j])
                    df['{}_{}_{}_{}_cumsum'.format(i,j,k,v)] = v_tmp['helper'].cumsum()
                    df['{}_{}_{}_{}_error_code_cumsum'.format(i,j,k,v)] = v_tmp['http_status_code_helper'].cumsum()

remove_cols = [x for x in df.columns if x[-6:]=='helper']

df = df.drop(columns=remove_cols)

### 保存特征

In [12]:
remove_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
       'browser_version', 'browser', 'os_type', 'os_version', 'op_datetime',
       'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url',
       'op_month', 'op_days', 'op_times_groups',
       'system_op_times_groups']

df = df.drop(columns=remove_cols)

In [16]:
df_train_cumsum = df[df[y_label].notna()].reset_index(drop=True)
df_val_cumsum = df[df[y_label].isna()].reset_index(drop=True)


save_pkl(df_train_cumsum.drop(columns=[y_label]), path_new_train_cumsum)
save_pkl(df_val_cumsum.drop(columns=[y_label]), path_new_test_cumsum)

## modeling

### 交叉累计特征+lightgbm

#### 获取建模使用的特征

In [18]:
remove_cols = ['id']
feats = df_train_cumsum.columns
feats = feats.drop(remove_cols)
feats = feats.drop(y_label)

In [19]:
df_train = df_train_cumsum.reset_index(drop=True)
df_test  = df_val_cumsum.reset_index(drop=True)

#### lightgbm

In [21]:
# feats = feats_importance.sort_values('importance', ascending=False)[:50].name.values

In [20]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

res_sort_col = 'id'

In [22]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []


for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        # 数据集构造
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label]
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label]
                         )
        # 模型训练
        model = lgb.train(params, train, valid_sets=[train, val], 
                          num_boost_round=20000, early_stopping_rounds=100, verbose_eval=10)
        
        # 结果及得分获取
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))

# 特征重要性获取
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

# 训练集得分效果
df_train['oof'] = oof
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

# 测试集合结果输出
score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values(res_sort_col).reset_index(drop=True)

if not os.path.exists(path_results_version):
    os.makedirs(path_results_version)

# 结果保存
sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_version,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.936921	valid_1's auc: 0.933065
[20]	training's auc: 0.940848	valid_1's auc: 0.936735
[30]	training's auc: 0.944431	valid_1's auc: 0.936934
[40]	training's auc: 0.947563	valid_1's auc: 0.936895
[50]	training's auc: 0.952719	valid_1's auc: 0.936269
[60]	training's auc: 0.955713	valid_1's auc: 0.936068
[70]	training's auc: 0.963375	valid_1's auc: 0.93683
[80]	training's auc: 0.968817	valid_1's auc: 0.935946
[90]	training's auc: 0.972562	valid_1's auc: 0.93549
[100]	training's auc: 0.975932	valid_1's auc: 0.935684
[110]	training's auc: 0.978344	valid_1's auc: 0.935428
[120]	training's auc: 0.979921	valid_1's auc: 0.935179
[130]	training's auc: 0.981753	valid_1's auc: 0.934588
[140]	training's auc: 0.983296	valid_1's auc: 0.934692
Early stopping, best iteration is:
[49]	training's auc: 0.951138	valid_1's auc: 0.93762
----------- 1
Training until validation scores don't improve for 100 rounds
[

,name,importance
66,system_system_op_times_groups_http_status_code...,97448.007593
1371,op_month_ip_transform_cumsum,23556.553337
519,op_times_groups_ip_transform_ip_type_内网_cumsum,20435.692204
219,system_op_days_http_status_code_200_cumsum,16073.540945
61,system_system_op_times_groups_ip_type_内网_cumsum,7271.235077
1067,op_days_user_name_cumsum,7062.083901
1431,op_month_ip_transform_ip_type_内网_cumsum,5953.030146
1436,op_month_ip_transform_http_status_code_200_cumsum,4872.665429
214,system_op_days_ip_type_内网_cumsum,3368.440034
576,op_times_groups_ip_transform_log_system_transf...,3145.646608


训练集AUC均值：0.9386216031892072, 训练集合AUC方差: 0.001409275104703724


### 交叉累计特征+分箱+lightgbm

#### 分箱

In [25]:
cols = df_train_cumsum.columns
cols = cols.drop(['id', y_label])

ord_encoder = ce.WOEEncoder(cols=cols)

df_train_cumsum_woe = ord_encoder.fit_transform(df_train_cumsum, df_train_cumsum[y_label] )
df_val_cumsum_woe  = ord_encoder.transform(df_val_cumsum)


display(df_train_cumsum_woe.head())
display(df_val_cumsum_woe.head())


save_pkl(df_train_cumsum_woe.drop(columns=[y_label]), path_new_train_cumsum_woe)
save_pkl(df_val_cumsum_woe.drop(columns=[y_label]), path_new_test_cumsum_woe)

,id,is_risk,system_system_op_times_groups_cumsum,system_system_op_times_groups_error_code_cumsum,system_system_op_times_groups_ip_transform_cumunique,system_system_op_times_groups_user_name_cumunique,system_system_op_times_groups_device_num_transform_cumunique,system_system_op_times_groups_department_cumunique,system_system_op_times_groups_department_rd_cumsum,system_system_op_times_groups_department_rd_error_code_cumsum,system_system_op_times_groups_department_hr_cumsum,system_system_op_times_groups_department_hr_error_code_cumsum,system_system_op_times_groups_department_sales_cumsum,system_system_op_times_groups_department_sales_error_code_cumsum,system_system_op_times_groups_department_other_cumsum,system_system_op_times_groups_department_other_error_code_cumsum,system_system_op_times_groups_department_nan_cumsum,system_system_op_times_groups_department_nan_error_code_cumsum,system_system_op_times_groups_department_accounting_cumsum,system_system_op_times_groups_department_accounting_error_code_cumsum,system_system_op_times_groups_browser_version_cumunique,system_system_op_times_groups_browser_version_chrome_93_cumsum,system_system_op_times_groups_browser_version_chrome_93_error_code_cumsum,system_system_op_times_groups_browser_version_safari_13_cumsum,system_system_op_times_groups_browser_version_safari_13_error_code_cumsum,system_system_op_times_groups_browser_version_firefox_78_cumsum,system_system_op_times_groups_browser_version_firefox_78_error_code_cumsum,system_system_op_times_groups_browser_version_edge_93_cumsum,system_system_op_times_groups_browser_version_edge_93_error_code_cumsum,system_system_op_times_groups_browser_version_ie_9_cumsum,system_system_op_times_groups_browser_version_ie_9_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_77_cumsum,system_system_op_times_groups_browser_version_chrome_77_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_90_cumsum,system_system_op_times_groups_browser_version_chrome_90_error_code_cumsum,system_system_op_times_groups_browser_version_ie_11_cumsum,system_system_op_times_groups_browser_version_ie_11_error_code_cumsum,system_system_op_times_groups_browser_cumunique,system_system_op_times_groups_browser_chrome_cumsum,system_system_op_times_groups_browser_chrome_error_code_cumsum,system_system_op_times_groups_browser_safari_cumsum,system_system_op_times_groups_browser_safari_error_code_cumsum,system_system_op_times_groups_browser_firefox_cumsum,system_system_op_times_groups_browser_firefox_error_code_cumsum,system_system_op_times_groups_browser_edge_cumsum,system_system_op_times_groups_browser_edge_error_code_cumsum,system_system_op_times_groups_browser_ie_cumsum,system_system_op_times_groups_browser_ie_error_code_cumsum,system_system_op_times_groups_os_type_cumunique,system_system_op_times_groups_os_type_win_cumsum,...,op_month_device_num_transform_op_city_苏州_cumsum,op_month_device_num_transform_op_city_苏州_error_code_cumsum,op_month_device_num_transform_log_system_transform_cumunique,op_month_device_num_transform_log_system_transform_nHrKgKdJ1Mzt_cumsum,op_month_device_num_transform_log_system_transform_nHrKgKdJ1Mzt_error_code_cumsum,op_month_device_num_transform_log_system_transform_a5G25puBl9xj_cumsum,op_month_device_num_transform_log_system_transform_a5G25puBl9xj_error_code_cumsum,op_month_device_num_transform_log_system_transform_sW0whYIx8LFM_cumsum,op_month_device_num_transform_log_system_transform_sW0whYIx8LFM_error_code_cumsum,op_month_device_num_transform_log_system_transform_2umVQwhiiwNJ_cumsum,op_month_device_num_transform_log_system_transform_2umVQwhiiwNJ_error_code_cumsum,op_month_device_num_transform_log_system_transform_fwM6KZKjrzjm_cumsum,op_month_device_num_transform_log_system_transform_fwM6KZKjrzjm_error_code_cumsum,op_month_device_num_transform_log_system_transform_nan_cumsum,op_month_device_num_transform_log_system_transform_nan_error_code_cumsum,op_month_device_num_transform_log_system_transform_dwS3cdn15GK4_cu

,id,is_risk,system_system_op_times_groups_cumsum,system_system_op_times_groups_error_code_cumsum,system_system_op_times_groups_ip_transform_cumunique,system_system_op_times_groups_user_name_cumunique,system_system_op_times_groups_device_num_transform_cumunique,system_system_op_times_groups_department_cumunique,system_system_op_times_groups_department_rd_cumsum,system_system_op_times_groups_department_rd_error_code_cumsum,system_system_op_times_groups_department_hr_cumsum,system_system_op_times_groups_department_hr_error_code_cumsum,system_system_op_times_groups_department_sales_cumsum,system_system_op_times_groups_department_sales_error_code_cumsum,system_system_op_times_groups_department_other_cumsum,system_system_op_times_groups_department_other_error_code_cumsum,system_system_op_times_groups_department_nan_cumsum,system_system_op_times_groups_department_nan_error_code_cumsum,system_system_op_times_groups_department_accounting_cumsum,system_system_op_times_groups_department_accounting_error_code_cumsum,system_system_op_times_groups_browser_version_cumunique,system_system_op_times_groups_browser_version_chrome_93_cumsum,system_system_op_times_groups_browser_version_chrome_93_error_code_cumsum,system_system_op_times_groups_browser_version_safari_13_cumsum,system_system_op_times_groups_browser_version_safari_13_error_code_cumsum,system_system_op_times_groups_browser_version_firefox_78_cumsum,system_system_op_times_groups_browser_version_firefox_78_error_code_cumsum,system_system_op_times_groups_browser_version_edge_93_cumsum,system_system_op_times_groups_browser_version_edge_93_error_code_cumsum,system_system_op_times_groups_browser_version_ie_9_cumsum,system_system_op_times_groups_browser_version_ie_9_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_77_cumsum,system_system_op_times_groups_browser_version_chrome_77_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_90_cumsum,system_system_op_times_groups_browser_version_chrome_90_error_code_cumsum,system_system_op_times_groups_browser_version_ie_11_cumsum,system_system_op_times_groups_browser_version_ie_11_error_code_cumsum,system_system_op_times_groups_browser_cumunique,system_system_op_times_groups_browser_chrome_cumsum,system_system_op_times_groups_browser_chrome_error_code_cumsum,system_system_op_times_groups_browser_safari_cumsum,system_system_op_times_groups_browser_safari_error_code_cumsum,system_system_op_times_groups_browser_firefox_cumsum,system_system_op_times_groups_browser_firefox_error_code_cumsum,system_system_op_times_groups_browser_edge_cumsum,system_system_op_times_groups_browser_edge_error_code_cumsum,system_system_op_times_groups_browser_ie_cumsum,system_system_op_times_groups_browser_ie_error_code_cumsum,system_system_op_times_groups_os_type_cumunique,system_system_op_times_groups_os_type_win_cumsum,...,op_month_device_num_transform_op_city_苏州_cumsum,op_month_device_num_transform_op_city_苏州_error_code_cumsum,op_month_device_num_transform_log_system_transform_cumunique,op_month_device_num_transform_log_system_transform_nHrKgKdJ1Mzt_cumsum,op_month_device_num_transform_log_system_transform_nHrKgKdJ1Mzt_error_code_cumsum,op_month_device_num_transform_log_system_transform_a5G25puBl9xj_cumsum,op_month_device_num_transform_log_system_transform_a5G25puBl9xj_error_code_cumsum,op_month_device_num_transform_log_system_transform_sW0whYIx8LFM_cumsum,op_month_device_num_transform_log_system_transform_sW0whYIx8LFM_error_code_cumsum,op_month_device_num_transform_log_system_transform_2umVQwhiiwNJ_cumsum,op_month_device_num_transform_log_system_transform_2umVQwhiiwNJ_error_code_cumsum,op_month_device_num_transform_log_system_transform_fwM6KZKjrzjm_cumsum,op_month_device_num_transform_log_system_transform_fwM6KZKjrzjm_error_code_cumsum,op_month_device_num_transform_log_system_transform_nan_cumsum,op_month_device_num_transform_log_system_transform_nan_error_code_cumsum,op_month_device_num_transform_log_system_transform_dwS3cdn15GK4_cu

#### 获取建模使用的特征

In [26]:
remove_cols = ['id']
feats = df_train_cumsum_woe.columns
feats = feats.drop(remove_cols)
feats = feats.drop(y_label)

In [27]:
df_train = df_train_cumsum_woe.reset_index(drop=True)
df_test  = df_val_cumsum_woe.reset_index(drop=True)

#### lightgbm

In [ ]:
# feats = feats_importance.sort_values('importance', ascending=False)[:50].name.values

In [28]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

res_sort_col = 'id'

In [29]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []


for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        # 数据集构造
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label]
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label]
                         )
        # 模型训练
        model = lgb.train(params, train, valid_sets=[train, val], 
                          num_boost_round=20000, early_stopping_rounds=100, verbose_eval=10)
        
        # 结果及得分获取
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))

# 特征重要性获取
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

# 训练集得分效果
df_train['oof'] = oof
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

# 测试集合结果输出
score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values(res_sort_col).reset_index(drop=True)

if not os.path.exists(path_results_version):
    os.makedirs(path_results_version)

# 结果保存
sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_version,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.997707	valid_1's auc: 0.99753
[20]	training's auc: 0.998437	valid_1's auc: 0.998152
[30]	training's auc: 0.998968	valid_1's auc: 0.998592
[40]	training's auc: 0.999218	valid_1's auc: 0.998795
[50]	training's auc: 0.999382	valid_1's auc: 0.999027
[60]	training's auc: 0.999533	valid_1's auc: 0.999135
[70]	training's auc: 0.999657	valid_1's auc: 0.999248
[80]	training's auc: 0.999759	valid_1's auc: 0.999312
[90]	training's auc: 0.999837	valid_1's auc: 0.99937
[100]	training's auc: 0.999897	valid_1's auc: 0.999419
[110]	training's auc: 0.999938	valid_1's auc: 0.999479
[120]	training's auc: 0.999966	valid_1's auc: 0.999531
[130]	training's auc: 0.999983	valid_1's auc: 0.999583
[140]	training's auc: 0.999991	valid_1's auc: 0.999605
[150]	training's auc: 0.999996	valid_1's auc: 0.999631
[160]	training's auc: 0.999998	valid_1's auc: 0.999647
[170]	training's auc: 0.999999	valid_1's auc: 0.999655


[220]	training's auc: 1	valid_1's auc: 0.999722
[230]	training's auc: 1	valid_1's auc: 0.999732
[240]	training's auc: 1	valid_1's auc: 0.999745
[250]	training's auc: 1	valid_1's auc: 0.999743
[260]	training's auc: 1	valid_1's auc: 0.999751
[270]	training's auc: 1	valid_1's auc: 0.999754
[280]	training's auc: 1	valid_1's auc: 0.999761
[290]	training's auc: 1	valid_1's auc: 0.999764
[300]	training's auc: 1	valid_1's auc: 0.999775
[310]	training's auc: 1	valid_1's auc: 0.999775
[320]	training's auc: 1	valid_1's auc: 0.999777
[330]	training's auc: 1	valid_1's auc: 0.999783
[340]	training's auc: 1	valid_1's auc: 0.999789
[350]	training's auc: 1	valid_1's auc: 0.999793
[360]	training's auc: 1	valid_1's auc: 0.999793
[370]	training's auc: 1	valid_1's auc: 0.999792
[380]	training's auc: 1	valid_1's auc: 0.999792
[390]	training's auc: 1	valid_1's auc: 0.999796
[400]	training's auc: 1	valid_1's auc: 0.999797
[410]	training's auc: 1	valid_1's auc: 0.999794
[420]	training's auc: 1	valid_1's auc: 0

[310]	training's auc: 1	valid_1's auc: 0.99969
[320]	training's auc: 1	valid_1's auc: 0.999694
[330]	training's auc: 1	valid_1's auc: 0.999695
[340]	training's auc: 1	valid_1's auc: 0.999695
[350]	training's auc: 1	valid_1's auc: 0.9997
[360]	training's auc: 1	valid_1's auc: 0.999699
[370]	training's auc: 1	valid_1's auc: 0.999699
[380]	training's auc: 1	valid_1's auc: 0.999701
[390]	training's auc: 1	valid_1's auc: 0.999701
[400]	training's auc: 1	valid_1's auc: 0.999705
[410]	training's auc: 1	valid_1's auc: 0.999708
[420]	training's auc: 1	valid_1's auc: 0.999705
[430]	training's auc: 1	valid_1's auc: 0.999707
[440]	training's auc: 1	valid_1's auc: 0.999712
[450]	training's auc: 1	valid_1's auc: 0.999714
[460]	training's auc: 1	valid_1's auc: 0.999715
[470]	training's auc: 1	valid_1's auc: 0.999714
[480]	training's auc: 1	valid_1's auc: 0.999716
[490]	training's auc: 1	valid_1's auc: 0.999721
[500]	training's auc: 1	valid_1's auc: 0.99972
[510]	training's auc: 1	valid_1's auc: 0.999

,name,importance
66,system_system_op_times_groups_http_status_code...,84310.406858
372,system_op_month_http_status_code_200_cumsum,57449.863679
306,system_op_month_cumsum,35115.960618
367,system_op_month_ip_type_内网_cumsum,15209.577616
312,system_op_month_department_rd_cumsum,14588.376152
676,op_times_groups_user_name_http_status_code_200...,10158.759267
1436,op_month_ip_transform_http_status_code_200_cumsum,10047.478490
1371,op_month_ip_transform_cumsum,8152.949611
219,system_op_days_http_status_code_200_cumsum,6912.536884
519,op_times_groups_ip_transform_ip_type_内网_cumsum,5474.440357


训练集AUC均值：0.9997557027280817, 训练集合AUC方差: 6.28066104293635e-05


### 类别特征+交叉累计特征+lightgbm

#### 类别特征+交叉累计特征

In [30]:
df_train_cate = read_pkl(path_new_train_cate)
df_val_cate = read_pkl(path_new_test_cate)

df_train_cumsum = read_pkl(path_new_train_cumsum)
df_val_cumsum = read_pkl(path_new_test_cumsum)

# df_train_cumsum_woe = read_pkl(path_new_train_cumsum_woe)
# df_val_cumsum_woe = read_pkl(path_new_test_cumsum_woe)

df_raw_train = pd.read_csv(path_train)
df_raw_val  = pd.read_csv(path_test)

In [31]:
df_row_train = pd.merge(left=df_train_cate, right=df_train_cumsum, on='id')
df_row_val = pd.merge(left=df_val_cate, right=df_val_cumsum, on='id')

df_row_train = pd.merge(left=df_row_train, right=df_raw_train[['id', y_label]], on='id')
df_row_val = pd.merge(left=df_row_val, right=df_raw_val[['id']], on='id')

In [32]:
df_row_train.head()

,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,ip_type,http_status_code,op_city,log_system_transform,url,system_system_op_times_groups_cumsum,system_system_op_times_groups_error_code_cumsum,system_system_op_times_groups_ip_transform_cumunique,system_system_op_times_groups_user_name_cumunique,system_system_op_times_groups_device_num_transform_cumunique,system_system_op_times_groups_department_cumunique,system_system_op_times_groups_department_rd_cumsum,system_system_op_times_groups_department_rd_error_code_cumsum,system_system_op_times_groups_department_hr_cumsum,system_system_op_times_groups_department_hr_error_code_cumsum,system_system_op_times_groups_department_sales_cumsum,system_system_op_times_groups_department_sales_error_code_cumsum,system_system_op_times_groups_department_other_cumsum,system_system_op_times_groups_department_other_error_code_cumsum,system_system_op_times_groups_department_nan_cumsum,system_system_op_times_groups_department_nan_error_code_cumsum,system_system_op_times_groups_department_accounting_cumsum,system_system_op_times_groups_department_accounting_error_code_cumsum,system_system_op_times_groups_browser_version_cumunique,system_system_op_times_groups_browser_version_chrome_93_cumsum,system_system_op_times_groups_browser_version_chrome_93_error_code_cumsum,system_system_op_times_groups_browser_version_safari_13_cumsum,system_system_op_times_groups_browser_version_safari_13_error_code_cumsum,system_system_op_times_groups_browser_version_firefox_78_cumsum,system_system_op_times_groups_browser_version_firefox_78_error_code_cumsum,system_system_op_times_groups_browser_version_edge_93_cumsum,system_system_op_times_groups_browser_version_edge_93_error_code_cumsum,system_system_op_times_groups_browser_version_ie_9_cumsum,system_system_op_times_groups_browser_version_ie_9_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_77_cumsum,system_system_op_times_groups_browser_version_chrome_77_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_90_cumsum,system_system_op_times_groups_browser_version_chrome_90_error_code_cumsum,system_system_op_times_groups_browser_version_ie_11_cumsum,system_system_op_times_groups_browser_version_ie_11_error_code_cumsum,system_system_op_times_groups_browser_cumunique,...,op_month_device_num_transform_op_city_苏州_error_code_cumsum,op_month_device_num_transform_log_system_transform_cumunique,op_month_device_num_transform_log_system_transform_nHrKgKdJ1Mzt_cumsum,op_month_device_num_transform_log_system_transform_nHrKgKdJ1Mzt_error_code_cumsum,op_month_device_num_transform_log_system_transform_a5G25puBl9xj_cumsum,op_month_device_num_transform_log_system_transform_a5G25puBl9xj_error_code_cumsum,op_month_device_num_transform_log_system_transform_sW0whYIx8LFM_cumsum,op_month_device_num_transform_log_system_transform_sW0whYIx8LFM_error_code_cumsum,op_month_device_num_transform_log_system_transform_2umVQwhiiwNJ_cumsum,op_month_device_num_transform_log_system_transform_2umVQwhiiwNJ_error_code_cumsum,op_month_device_num_transform_log_system_transform_fwM6KZKjrzjm_cumsum,op_month_device_num_transform_log_system_transform_fwM6KZKjrzjm_error_code_cumsum,op_month_device_num_transform_log_system_transform_nan_cumsum,op_month_device_num_transform_log_system_transform_nan_error_code_cumsum,op_month_device_num_transform_log_system_transform_dwS3cdn15GK4_cumsum,op_month_device_num_transform_log_system_transform_dwS3cdn15GK4_error_code_cumsum,op_month_device_num_transform_log_system_transform_RwHe8Q1R7AlB_cumsum,op_month_device_num_transform_log_system_transform_RwHe8Q1R7AlB_error_code_cumsum,op_month_device_num_transform_log_system_transform_9RAS6RNfETj5_cumsum,op_month_device_num_transform_log_system_transform_9RAS6RNfETj5_error_code_cumsum,op_month_device_num_transform_log_system_transform_2UNHLdxlhIzv_cumsum,op_month_device_num_transform_log_system_transform_2UNHLdxlhIzv_error_code_cumsum,op_month_device_

#### 获取建模使用的特征

In [33]:
remove_cols = ['id']
feats = df_row_train.columns
feats = feats.drop(remove_cols)
feats = feats.drop(y_label)

In [34]:
df_train = df_row_train.reset_index(drop=True)
df_test  = df_row_val.reset_index(drop=True)

#### lightgbm

In [35]:
# feats = feats_importance.sort_values('importance', ascending=False)[:50].name.values

In [36]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

res_sort_col = 'id'

In [37]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []


for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        # 数据集构造
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label]
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label]
                         )
        # 模型训练
        model = lgb.train(params, train, valid_sets=[train, val], 
                          num_boost_round=20000, early_stopping_rounds=100, verbose_eval=10)
        
        # 结果及得分获取
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))

# 特征重要性获取
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

# 训练集得分效果
df_train['oof'] = oof
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

# 测试集合结果输出
score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values(res_sort_col).reset_index(drop=True)

if not os.path.exists(path_results_version):
    os.makedirs(path_results_version)

# 结果保存
sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_version,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.943424	valid_1's auc: 0.937223
[20]	training's auc: 0.945581	valid_1's auc: 0.936662
[30]	training's auc: 0.94926	valid_1's auc: 0.938379
[40]	training's auc: 0.951674	valid_1's auc: 0.940209
[50]	training's auc: 0.956432	valid_1's auc: 0.939096
[60]	training's auc: 0.960539	valid_1's auc: 0.939226
[70]	training's auc: 0.964737	valid_1's auc: 0.941227
[80]	training's auc: 0.968534	valid_1's auc: 0.942068
[90]	training's auc: 0.972735	valid_1's auc: 0.942206
[100]	training's auc: 0.975309	valid_1's auc: 0.942521
[110]	training's auc: 0.97851	valid_1's auc: 0.942221
[120]	training's auc: 0.98072	valid_1's auc: 0.94219
[130]	training's auc: 0.98229	valid_1's auc: 0.94143
[140]	training's auc: 0.983721	valid_1's auc: 0.941108
[150]	training's auc: 0.98471	valid_1's auc: 0.94068
[160]	training's auc: 0.985906	valid_1's auc: 0.940311
[170]	training's auc: 0.987271	valid_1's auc: 0.939824
[180]	

,name,importance
79,system_system_op_times_groups_http_status_code...,86719.418591
1384,op_month_ip_transform_cumsum,20214.377991
532,op_times_groups_ip_transform_ip_type_内网_cumsum,18605.812027
232,system_op_days_http_status_code_200_cumsum,17734.585225
1449,op_month_ip_transform_http_status_code_200_cumsum,10083.128139
74,system_system_op_times_groups_ip_type_内网_cumsum,9878.142684
1444,op_month_ip_transform_ip_type_内网_cumsum,7645.039891
1080,op_days_user_name_cumsum,7443.708265
227,system_op_days_ip_type_内网_cumsum,3570.450950
589,op_times_groups_ip_transform_log_system_transf...,3477.343858


训练集AUC均值：0.9417167997112633, 训练集合AUC方差: 0.0016311908434587204


### 类别特征+交叉累计特征WOE+lightgbm

#### 类别特征+交叉累计特征WOE

In [38]:
df_train_cate = read_pkl(path_new_train_cate)
df_val_cate = read_pkl(path_new_test_cate)

df_train_cumsum_woe = read_pkl(path_new_train_cumsum_woe)
df_val_cumsum_woe = read_pkl(path_new_test_cumsum_woe)

df_raw_train = pd.read_csv(path_train)
df_raw_val  = pd.read_csv(path_test)

In [39]:
df_row_train = pd.merge(left=df_train_cate, right=df_train_cumsum_woe, on='id')
df_row_val = pd.merge(left=df_val_cate, right=df_val_cumsum_woe, on='id')

df_row_train = pd.merge(left=df_row_train, right=df_raw_train[['id', y_label]], on='id')
df_row_val = pd.merge(left=df_row_val, right=df_raw_val[['id']], on='id')

In [40]:
df_row_train.head()

,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,ip_type,http_status_code,op_city,log_system_transform,url,system_system_op_times_groups_cumsum,system_system_op_times_groups_error_code_cumsum,system_system_op_times_groups_ip_transform_cumunique,system_system_op_times_groups_user_name_cumunique,system_system_op_times_groups_device_num_transform_cumunique,system_system_op_times_groups_department_cumunique,system_system_op_times_groups_department_rd_cumsum,system_system_op_times_groups_department_rd_error_code_cumsum,system_system_op_times_groups_department_hr_cumsum,system_system_op_times_groups_department_hr_error_code_cumsum,system_system_op_times_groups_department_sales_cumsum,system_system_op_times_groups_department_sales_error_code_cumsum,system_system_op_times_groups_department_other_cumsum,system_system_op_times_groups_department_other_error_code_cumsum,system_system_op_times_groups_department_nan_cumsum,system_system_op_times_groups_department_nan_error_code_cumsum,system_system_op_times_groups_department_accounting_cumsum,system_system_op_times_groups_department_accounting_error_code_cumsum,system_system_op_times_groups_browser_version_cumunique,system_system_op_times_groups_browser_version_chrome_93_cumsum,system_system_op_times_groups_browser_version_chrome_93_error_code_cumsum,system_system_op_times_groups_browser_version_safari_13_cumsum,system_system_op_times_groups_browser_version_safari_13_error_code_cumsum,system_system_op_times_groups_browser_version_firefox_78_cumsum,system_system_op_times_groups_browser_version_firefox_78_error_code_cumsum,system_system_op_times_groups_browser_version_edge_93_cumsum,system_system_op_times_groups_browser_version_edge_93_error_code_cumsum,system_system_op_times_groups_browser_version_ie_9_cumsum,system_system_op_times_groups_browser_version_ie_9_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_77_cumsum,system_system_op_times_groups_browser_version_chrome_77_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_90_cumsum,system_system_op_times_groups_browser_version_chrome_90_error_code_cumsum,system_system_op_times_groups_browser_version_ie_11_cumsum,system_system_op_times_groups_browser_version_ie_11_error_code_cumsum,system_system_op_times_groups_browser_cumunique,...,op_month_device_num_transform_op_city_苏州_error_code_cumsum,op_month_device_num_transform_log_system_transform_cumunique,op_month_device_num_transform_log_system_transform_nHrKgKdJ1Mzt_cumsum,op_month_device_num_transform_log_system_transform_nHrKgKdJ1Mzt_error_code_cumsum,op_month_device_num_transform_log_system_transform_a5G25puBl9xj_cumsum,op_month_device_num_transform_log_system_transform_a5G25puBl9xj_error_code_cumsum,op_month_device_num_transform_log_system_transform_sW0whYIx8LFM_cumsum,op_month_device_num_transform_log_system_transform_sW0whYIx8LFM_error_code_cumsum,op_month_device_num_transform_log_system_transform_2umVQwhiiwNJ_cumsum,op_month_device_num_transform_log_system_transform_2umVQwhiiwNJ_error_code_cumsum,op_month_device_num_transform_log_system_transform_fwM6KZKjrzjm_cumsum,op_month_device_num_transform_log_system_transform_fwM6KZKjrzjm_error_code_cumsum,op_month_device_num_transform_log_system_transform_nan_cumsum,op_month_device_num_transform_log_system_transform_nan_error_code_cumsum,op_month_device_num_transform_log_system_transform_dwS3cdn15GK4_cumsum,op_month_device_num_transform_log_system_transform_dwS3cdn15GK4_error_code_cumsum,op_month_device_num_transform_log_system_transform_RwHe8Q1R7AlB_cumsum,op_month_device_num_transform_log_system_transform_RwHe8Q1R7AlB_error_code_cumsum,op_month_device_num_transform_log_system_transform_9RAS6RNfETj5_cumsum,op_month_device_num_transform_log_system_transform_9RAS6RNfETj5_error_code_cumsum,op_month_device_num_transform_log_system_transform_2UNHLdxlhIzv_cumsum,op_month_device_num_transform_log_system_transform_2UNHLdxlhIzv_error_code_cumsum,op_month_device_

#### 获取建模使用的特征

In [41]:
remove_cols = ['id']
feats = df_row_train.columns
feats = feats.drop(remove_cols)
feats = feats.drop(y_label)

In [42]:
df_train = df_row_train.reset_index(drop=True)
df_test  = df_row_val.reset_index(drop=True)

#### lightgbm

In [43]:
# feats = feats_importance.sort_values('importance', ascending=False)[:50].name.values

In [44]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

res_sort_col = 'id'

In [45]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []


for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        # 数据集构造
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label]
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label]
                         )
        # 模型训练
        model = lgb.train(params, train, valid_sets=[train, val], 
                          num_boost_round=20000, early_stopping_rounds=100, verbose_eval=10)
        
        # 结果及得分获取
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))

# 特征重要性获取
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

# 训练集得分效果
df_train['oof'] = oof
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

# 测试集合结果输出
score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values(res_sort_col).reset_index(drop=True)

if not os.path.exists(path_results_version):
    os.makedirs(path_results_version)

# 结果保存
sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_version,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.997875	valid_1's auc: 0.996358
[20]	training's auc: 0.998534	valid_1's auc: 0.997257
[30]	training's auc: 0.998962	valid_1's auc: 0.998126
[40]	training's auc: 0.999254	valid_1's auc: 0.998593
[50]	training's auc: 0.999422	valid_1's auc: 0.998805
[60]	training's auc: 0.999559	valid_1's auc: 0.998964
[70]	training's auc: 0.999692	valid_1's auc: 0.999091
[80]	training's auc: 0.999779	valid_1's auc: 0.999187
[90]	training's auc: 0.999857	valid_1's auc: 0.99927
[100]	training's auc: 0.99991	valid_1's auc: 0.999302
[110]	training's auc: 0.999946	valid_1's auc: 0.999342
[120]	training's auc: 0.99997	valid_1's auc: 0.999377
[130]	training's auc: 0.999984	valid_1's auc: 0.999416
[140]	training's auc: 0.999993	valid_1's auc: 0.999445
[150]	training's auc: 0.999997	valid_1's auc: 0.999475
[160]	training's auc: 0.999999	valid_1's auc: 0.999506
[170]	training's auc: 1	valid_1's auc: 0.999516
[180]	tr

[40]	training's auc: 0.999257	valid_1's auc: 0.998513
[50]	training's auc: 0.999439	valid_1's auc: 0.99878
[60]	training's auc: 0.999552	valid_1's auc: 0.9989
[70]	training's auc: 0.999653	valid_1's auc: 0.998964
[80]	training's auc: 0.999746	valid_1's auc: 0.998986
[90]	training's auc: 0.999836	valid_1's auc: 0.999029
[100]	training's auc: 0.999902	valid_1's auc: 0.999066
[110]	training's auc: 0.999946	valid_1's auc: 0.999185
[120]	training's auc: 0.999971	valid_1's auc: 0.999204
[130]	training's auc: 0.999985	valid_1's auc: 0.999256
[140]	training's auc: 0.999993	valid_1's auc: 0.999294
[150]	training's auc: 0.999997	valid_1's auc: 0.999312
[160]	training's auc: 0.999999	valid_1's auc: 0.999348
[170]	training's auc: 1	valid_1's auc: 0.999362
[180]	training's auc: 1	valid_1's auc: 0.999381
[190]	training's auc: 1	valid_1's auc: 0.999397
[200]	training's auc: 1	valid_1's auc: 0.999406
[210]	training's auc: 1	valid_1's auc: 0.999415
[220]	training's auc: 1	valid_1's auc: 0.999423
[230]	

Early stopping, best iteration is:
[608]	training's auc: 1	valid_1's auc: 0.999789


,name,importance
79,system_system_op_times_groups_http_status_code...,70250.894281
385,system_op_month_http_status_code_200_cumsum,60643.254283
319,system_op_month_cumsum,34731.935921
380,system_op_month_ip_type_内网_cumsum,16218.417339
325,system_op_month_department_rd_cumsum,15000.533932
1449,op_month_ip_transform_http_status_code_200_cumsum,13608.113620
689,op_times_groups_user_name_http_status_code_200...,12334.220365
1384,op_month_ip_transform_cumsum,8895.416552
232,system_op_days_http_status_code_200_cumsum,7402.204626
371,system_op_month_os_version_win10_cumsum,5462.277851


训练集AUC均值：0.9997050271838572, 训练集合AUC方差: 0.00013631547061740645
